In [127]:
import os
import multiprocessing
import time

In [124]:
from multiprocessing import Process

In [129]:
def f(name):
    time.sleep(5)
    print('hello', name)

In [130]:
f('mat')
f('bob')

hello mat
hello bob


In [133]:
ps = []
ps.append(Process(target=f, args=('bob',)))
ps.append(Process(target=f, args=('mat',)))
ps.append(Process(target=f, args=('jay',)))
for i in ps:
    i.start()

hello bob
hello mat
hello jay


In [121]:
os.system("python3 train.py 'data/India/IndusINDVal.csv' 'data/India/IndusINDVal.csv' --strategy='t-dqn' --model-name='IndusInd'")

2

# Visualization: Trading Session

In [4]:
import pandas as pd
import numpy as np

import altair as alt
import seaborn as sns

### 1. Define parameters and Load model

In [105]:
from trading_bot.agent import Agent

model_name = 'model_dqn_GOOG_50'
test_stock = 'data/GOOG_2019.csv'
window_size = 10
debug = True

agent = Agent(window_size, pretrained=True, model_name=model_name)

### 2. Load test data

In [106]:
# read csv into dataframe
df = pd.read_csv(test_stock)
# filter out the desired features
df = df[['Date', 'Adj Close']]
# rename feature column names
df = df.rename(columns={'Adj Close': 'actual', 'Date': 'date'})
# convert dates from object to DateTime type
dates = df['date']
dates = pd.to_datetime(dates, infer_datetime_format=True)
df['date'] = dates

df.head()

,date,actual
0,2019-01-02,1045.849976
1,2019-01-03,1016.059998
2,2019-01-04,1070.709961
3,2019-01-07,1068.390015
4,2019-01-08,1076.280029


### 3. Running Eval

In [107]:
import logging
import coloredlogs

from trading_bot.utils import show_eval_result, switch_k_backend_device, get_stock_data
from trading_bot.methods import evaluate_model

coloredlogs.install(level='DEBUG')
switch_k_backend_device()

test_data = get_stock_data(test_stock)
initial_offset = test_data[1] - test_data[0]

test_result, history = evaluate_model(agent, test_data, window_size, debug)
show_eval_result(model_name, test_result, initial_offset)

2020-09-25 15:56:27 Tornado root[2961] DEBUG switching to TensorFlow for CPU
2020-09-25 15:56:27 Tornado root[2961] DEBUG Buy at: $1045.85
2020-09-25 15:56:27 Tornado root[2961] DEBUG Sell at: $1016.06 | Position: -$29.79
2020-09-25 15:56:27 Tornado root[2961] DEBUG Buy at: $1057.19
2020-09-25 15:56:27 Tornado root[2961] DEBUG Buy at: $1080.97
2020-09-25 15:56:27 Tornado root[2961] DEBUG Sell at: $1089.90 | Position: +$32.71
2020-09-25 15:56:27 Tornado root[2961] DEBUG Sell at: $1098.26 | Position: +$17.29
2020-09-25 15:56:27 Tornado root[2961] DEBUG Buy at: $1073.90
2020-09-25 15:56:27 Tornado root[2961] DEBUG Sell at: $1090.99 | Position: +$17.09
2020-09-25 15:56:27 Tornado root[2961] DEBUG Buy at: $1098.71
2020-09-25 15:56:27 Tornado root[2961] DEBUG Buy at: $1095.06
2020-09-25 15:56:27 Tornado root[2961] DEBUG Buy at: $1095.01
2020-09-25 15:56:27 Tornado root[2961] DEBUG Sell at: $1121.37 | Position: +$22.66
2020-09-25 15:56:27 Tornado root[2961] DEBUG Buy at: $1120.16
2020-09-25 1

2020-09-25 15:56:28 Tornado root[2961] DEBUG Sell at: $1188.10 | Position: +$23.81
2020-09-25 15:56:28 Tornado root[2961] DEBUG Sell at: $1168.39 | Position: +$1.13
2020-09-25 15:56:28 Tornado root[2961] DEBUG Sell at: $1181.41 | Position: +$3.81
2020-09-25 15:56:28 Tornado root[2961] DEBUG Sell at: $1211.38 | Position: +$21.85
2020-09-25 15:56:28 Tornado root[2961] DEBUG Sell at: $1204.93 | Position: +$36.04
2020-09-25 15:56:28 Tornado root[2961] DEBUG Buy at: $1204.41
2020-09-25 15:56:28 Tornado root[2961] DEBUG Sell at: $1206.00 | Position: +$1.59
2020-09-25 15:56:28 Tornado root[2961] DEBUG Buy at: $1229.15
2020-09-25 15:56:28 Tornado root[2961] DEBUG Sell at: $1232.41 | Position: +$3.26
2020-09-25 15:56:28 Tornado root[2961] DEBUG Buy at: $1225.09
2020-09-25 15:56:28 Tornado root[2961] DEBUG Sell at: $1219.00 | Position: -$6.09
2020-09-25 15:56:28 Tornado root[2961] DEBUG Buy at: $1209.00
2020-09-25 15:56:28 Tornado root[2961] DEBUG Sell at: $1207.68 | Position: -$1.32
2020-09-25 

### 4. Visualize

In [108]:
def visualize(df, history, title="trading session"):
    # add history to dataframe
    position = [history[0][0]] + [x[0] for x in history]
    actions = ['HOLD'] + [x[1] for x in history]
    df['position'] = position
    df['action'] = actions
    
    # specify y-axis scale for stock prices
    scale = alt.Scale(domain=(min(min(df['actual']), min(df['position'])) - 50, max(max(df['actual']), max(df['position'])) + 50), clamp=True)
    
    # plot a line chart for stock positions
    actual = alt.Chart(df).mark_line(
        color='green',
        opacity=0.5
    ).encode(
        x='date:T',
        y=alt.Y('position', axis=alt.Axis(format='$.2f', title='Price'), scale=scale)
    ).interactive(
        bind_y=False
    )
    
    # plot the BUY and SELL actions as points
    points = alt.Chart(df).transform_filter(
        alt.datum.action != 'HOLD'
    ).mark_point(
        filled=True
    ).encode(
        x=alt.X('date:T', axis=alt.Axis(title='Date')),
        y=alt.Y('position', axis=alt.Axis(format='$.2f', title='Price'), scale=scale),
        color='action'
    ).interactive(bind_y=False)

    # merge the two charts
    chart = alt.layer(actual, points, title=title).properties(height=300, width=1000)
    
    return chart

In [109]:
chart = visualize(df, history, title=test_stock)
chart

alt.LayerChart(...)

In [110]:
df.iloc[:25]

,date,actual,position,action
0,2019-01-02,1045.849976,1045.849976,HOLD
1,2019-01-03,1016.059998,1045.849976,BUY
2,2019-01-04,1070.709961,1016.059998,SELL
3,2019-01-07,1068.390015,1070.709961,HOLD
4,2019-01-08,1076.280029,1068.390015,HOLD
5,2019-01-09,1074.660034,1076.280029,HOLD
6,2019-01-10,1070.329956,1074.660034,HOLD
7,2019-01-11,1057.189941,1070.329956,HOLD
8,2019-01-14,1044.689941,1057.189941,BUY
9,2019-01-15,1077.150024,1044.689941,HOLD


In [111]:
cap = [0]
inv = 0
ret = 0
b = 0
for i in range(len(df)):
    if df.iloc[i]['action']=='BUY':
        cap.append(cap[-1]+df.iloc[i]['actual'])
        inv+=df.iloc[i]['actual']
        b+=1
    if df.iloc[i]['action']=='SELL' and b>0:
        cap.append(cap[-1]-df.iloc[i]['actual'])
        ret += df.iloc[i]['actual']
        b-=1

In [112]:
req_cap = max(cap)
req_cap

14039.629818999996

In [113]:
prof = ret+(df['action'].value_counts()['BUY']-df['action'].value_counts()['SELL'])*df.iloc[25]['actual']-inv

In [114]:
(prof/ req_cap)*100

10.687318144025465

In [115]:
df['action'].value_counts()

HOLD    97
BUY     77
SELL    77
Name: action, dtype: int64

In [118]:
position = [history[0][0]] + [x[0] for x in history]

In [116]:
df

,date,actual,position,action
0,2019-01-02,1045.849976,1045.849976,HOLD
1,2019-01-03,1016.059998,1045.849976,BUY
2,2019-01-04,1070.709961,1016.059998,SELL
3,2019-01-07,1068.390015,1070.709961,HOLD
4,2019-01-08,1076.280029,1068.390015,HOLD
5,2019-01-09,1074.660034,1076.280029,HOLD
6,2019-01-10,1070.329956,1074.660034,HOLD
7,2019-01-11,1057.189941,1070.329956,HOLD
8,2019-01-14,1044.689941,1057.189941,BUY
9,2019-01-15,1077.150024,1044.689941,HOLD


4233.569336